In [1]:
## Optional
# Format output of Jupyter Notebook
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Hide python warnings
import warnings
warnings.filterwarnings('ignore')

In [130]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql.functions import lit, isnull, when, count, col, regexp_extract, concat_ws, to_date, expr, quarter, when, date_add, year, month, day, dayofweek

# TODO - parameterize endpoints/ports such as minio:9000 with envs

# Define config for SparkSession,
# such as Iceberg catalog that utilizes minio, an S3-compatible local object storage
spark_configs = {
    'spark.master': 'spark://spark-iceberg:7077',
    'spark.sql.catalog.airline': 'org.apache.iceberg.spark.SparkCatalog',
    # 'spark.sql.catalog.airline': 'org.apache.iceberg.spark.SparkSessionCatalog',
    'spark.sql.catalog.airline.io-impl': 'org.apache.iceberg.aws.s3.S3FileIO',
    'spark.sql.catalog.airline.s3.endpoint': 'http://minio:9000',
    'spark.sql.catalog.airline.type': 'rest',
    'spark.sql.catalog.airline.uri': 'http://rest:8181',
    'spark.sql.catalog.airline.warehouse': 's3://warehouse',
    'spark.sql.defaultCatalog': 'airline'
}

# Initialize SparkSession
spark = (
    SparkSession
    .builder
    .appName('EDA')
    .config(map=spark_configs)
    .getOrCreate()
)

In [7]:
import pprint as pp

# print(spark.sparkContext.uiWebUrl)

print(spark.conf.get('spark.app.id'))
print(spark.conf.get('spark.app.name'))
print(spark.conf.get('spark.app.startTime'))
print(spark.conf.get('spark.master'))
print()
print(spark.conf.get('spark.sql.catalog.airline'))
print(spark.conf.get('spark.sql.catalog.airline.io-impl'))
print(spark.conf.get('spark.sql.catalog.airline.s3.endpoint'))
print(spark.conf.get('spark.sql.catalog.airline.type'))
print(spark.conf.get('spark.sql.catalog.airline.uri'))
print(spark.conf.get('spark.sql.catalog.airline.warehouse'))
print()

                     
# pp.pp(dict(sc.getConf().getAll()), sort_dicts=True, )

local-1734722159949
EDA
1734722158776
spark://spark-iceberg:7077

org.apache.iceberg.spark.SparkCatalog
org.apache.iceberg.aws.s3.S3FileIO
http://minio:9000
rest
http://rest:8181
s3://warehouse



# Ingest raw csv data

In [37]:
# Check for alphabetic characters in flight_number, this is useless...
raw_data = spark.read.option('header', True).csv(filename)
raw_data.where(regexp_extract('flight_number', '^\d+$', 0) == '').show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [5]:
# Ingest flights data

# TODO: Fix nullables column, cant set column as Not-Nullable for some reason...

# Define schema
flights_schema = StructType([
    StructField("year", IntegerType(), False),   # Non-Nullable
    StructField("month", IntegerType(), False),  # Non-Nullable
    StructField("day", IntegerType(), False),    # Non-Nullable
    StructField("day_of_week", IntegerType(), False),    # Non-Nullable
    StructField("airline", StringType(), False),         # Non-Nullable
    StructField("flight_number", IntegerType(), False),  # Non-Nullable
    StructField("tail_number", StringType(), True),
    StructField("origin_airport", StringType(), False),  # Non-Nullable
    StructField("destination_airport", StringType(), False),   # Non-Nullable
    StructField("scheduled_departure", IntegerType(), False),  # Non-Nullable
    StructField("departure_time", IntegerType(), True),
    StructField("departure_delay", IntegerType(), True),
    StructField("taxi_out", IntegerType(), True),
    StructField("wheels_off", IntegerType(), True),
    StructField("scheduled_time", IntegerType(), True),
    StructField("elapsed_time", IntegerType(), True),
    StructField("air_time", IntegerType(), True),
    StructField("distance", IntegerType(), False),  # Non-Nullable
    StructField("wheels_on", IntegerType(), True),
    StructField("taxi_in", IntegerType(), True),
    StructField("scheduled_arrival", IntegerType(), False),  # Non-Nullable
    StructField("arrival_time", IntegerType(), True),
    StructField("arrival_delay", IntegerType(), True),
    StructField("diverted", IntegerType(), False),   # Non-Nullable
    StructField("cancelled", IntegerType(), False),  # Non-Nullable
    StructField("cancellation_reason", StringType(), True),
    StructField("air_system_delay", IntegerType(), True),
    StructField("security_delay", IntegerType(), True),
    StructField("airline_delay", IntegerType(), True),
    StructField("late_aircraft_delay", IntegerType(), True),
    StructField("weather_delay", IntegerType(), True)
])


# Read flights.csv
filename = '/home/iceberg/data/flights.csv'
flights_df = spark.read.schema(flights_schema).csv(
    '/home/iceberg/data/flights.csv',
    schema=flights_schema,
    enforceSchema=True,
    header = True
)
flights_df = flights_df.sort(['year', 'month', 'day', 'scheduled_departure'])
flights_df.cache()

flights_df.show()
# flights_df.printSchema()
flights_df.explain()

24/12/21 05:10:27 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|year|month|day|day_of_week|airline|flight_number|tail_number|origin_airport|destination_airport|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [ ]:
# Experimenting

# flights_df.groupBy(['flight_number', 'tail_number']).count().show()
# flights_df.filter(col('flight_number') == 2519).sort(['airline', 'month', 'day']).show(50)

# # Spark SQL
# date_df = flights_df.select(['year', 'month', 'day', 'day_of_week']).withColumn('date', expr('to_date(concat(year, "-", month))'))

In [135]:
## Create dim_date table
# Create date_df, we extract the date info from flights_df and dedup,
# Then add columns for date and quarter
date_df = flights_df \
    .select(['year', 'month', 'day', 'day_of_week']) \
    .dropDuplicates(['year', 'month', 'day', 'day_of_week']) \
    .withColumn('date', to_date(concat_ws('-', 'year', 'month', 'day'))) \
    .withColumn('quarter', quarter('date'))
date_df.cache()
# date_df.show()

# List of U.S. federal holidays
us_holidays_2015 = [
    ("2015-01-01", "New Year's Day"),
    ("2015-01-19", "Martin Luther King Jr. Day"),
    ("2015-02-16", "Presidents' Day"),
    ("2015-05-25", "Memorial Day"),
    ("2015-07-04", "Independence Day"),
    ("2015-09-07", "Labor Day"),
    ("2015-10-12", "Columbus Day"),
    ("2015-11-11", "Veterans Day"),
    ("2015-11-26", "Thanksgiving Day"),
    ("2015-12-25", "Christmas Day"),
]

# Create holidays_df and cast date from STRING to DATE type
holidays_df = spark.createDataFrame(us_holidays_2015, ['holiday_date', 'holiday_name'])
holidays_df = holidays_df.withColumn('holiday_date', to_date('holiday_date'))

# Join holidays to date_df and add is_holiday column
date_df = date_df \
    .join(
        holidays_df,
        date_df.date == holidays_df.holiday_date,
        'left'
    ) \
    .withColumn(
        'is_holiday',
        when(col('holiday_name').isNotNull(), lit(True)).otherwise(lit(False))
    ) \
    .drop('holiday_date') \
    .sort('date')

# Rearrange date to be first column,
date_df = date_df.select('date', *[col(c) for c in date_df.columns if c != 'date'])

date_df.show()

24/12/21 06:59:36 WARN CacheManager: Asked to cache already cached data.


+----------+----+-----+---+-----------+-------+--------------------+----------+
|      date|year|month|day|day_of_week|quarter|        holiday_name|is_holiday|
+----------+----+-----+---+-----------+-------+--------------------+----------+
|2015-01-01|2015|    1|  1|          4|      1|      New Year's Day|      true|
|2015-01-02|2015|    1|  2|          5|      1|                NULL|     false|
|2015-01-03|2015|    1|  3|          6|      1|                NULL|     false|
|2015-01-04|2015|    1|  4|          7|      1|                NULL|     false|
|2015-01-05|2015|    1|  5|          1|      1|                NULL|     false|
|2015-01-06|2015|    1|  6|          2|      1|                NULL|     false|
|2015-01-07|2015|    1|  7|          3|      1|                NULL|     false|
|2015-01-08|2015|    1|  8|          4|      1|                NULL|     false|
|2015-01-09|2015|    1|  9|          5|      1|                NULL|     false|
|2015-01-10|2015|    1| 10|          6| 

In [139]:
# Original day_of_week 1 is Monday, spark dayofweek 1 is Sunday
date_df = spark.range(365) \
    .withColumn('date', expr('date_add("2015-01-01", CAST(id AS INT))')) \
    .withColumn('year', year('date')) \
    .withColumn('month', month('date')) \
    .withColumn('day', day('date')) \
    .withColumn('day_of_week', dayofweek('date')) \
    .withColumn('quarter', quarter('date')) \
    .drop('id')

date_df.show()

+----------+----+-----+---+-----------+-------+
|      date|year|month|day|day_of_week|quarter|
+----------+----+-----+---+-----------+-------+
|2015-01-01|2015|    1|  1|          5|      1|
|2015-01-02|2015|    1|  2|          6|      1|
|2015-01-03|2015|    1|  3|          7|      1|
|2015-01-04|2015|    1|  4|          1|      1|
|2015-01-05|2015|    1|  5|          2|      1|
|2015-01-06|2015|    1|  6|          3|      1|
|2015-01-07|2015|    1|  7|          4|      1|
|2015-01-08|2015|    1|  8|          5|      1|
|2015-01-09|2015|    1|  9|          6|      1|
|2015-01-10|2015|    1| 10|          7|      1|
|2015-01-11|2015|    1| 11|          1|      1|
|2015-01-12|2015|    1| 12|          2|      1|
|2015-01-13|2015|    1| 13|          3|      1|
|2015-01-14|2015|    1| 14|          4|      1|
|2015-01-15|2015|    1| 15|          5|      1|
|2015-01-16|2015|    1| 16|          6|      1|
|2015-01-17|2015|    1| 17|          7|      1|
|2015-01-18|2015|    1| 18|          1| 

# Explore data!
TODO:
- Check statistics of important columns, like what are the percentiles for departure/arrival delay?
- Check max values for delay

## Total number of flight records: 5,819,079

In [32]:
flights_df.count()

5819079

## Total number of cancelled flights: 89,884, Total number of diverted flights: 15,187
These will cause nulls in other columns.

In [37]:
flights_df.where('cancelled = 1').count()

89884

In [35]:
flights_df.where('diverted = 1').count()

15187

## Get Null counts for each column

In [31]:
flights_df.select([count(when(isnull(c), c)).alias(c) for c in data.columns]).show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

## Investigate some causes for nulls in each column

In [48]:
# List of columns with null values
null_columns = [col for col in flights_df.columns if flights_df.filter(f"{col} IS NULL").count() > 0]

# Iterate through columns to investigate null causes
for col in null_columns:
    print(f"Investigating nulls in column: {col}")
    # Count rows where column is null and cancelled == 1
    count = flights_df.where(f"`{col}` IS NULL AND cancelled = 1").count()
    print(f"Nulls in {col} where flight is cancelled: {count}\n")

Investigating nulls in column: tail_number


Nulls in tail_number where flight is cancelled: 14721

Investigating nulls in column: departure_time


Nulls in departure_time where flight is cancelled: 86153

Investigating nulls in column: departure_delay


Nulls in departure_delay where flight is cancelled: 86153

Investigating nulls in column: taxi_out


Nulls in taxi_out where flight is cancelled: 89047

Investigating nulls in column: wheels_off


...

Nulls in air_system_delay where flight is cancelled: 89884

Investigating nulls in column: security_delay


Nulls in security_delay where flight is cancelled: 89884

Investigating nulls in column: airline_delay


Nulls in airline_delay where flight is cancelled: 89884

Investigating nulls in column: late_aircraft_delay


Nulls in late_aircraft_delay where flight is cancelled: 89884

Investigating nulls in column: weather_delay


Nulls in weather_delay where flight is cancelled: 89884



#### Tail_number - all Nulls are from cancelled flights
Although, not all cancelled flights have Nulls in tail_number. Possibly some flights are cancelled before an aircraft is assigned to the flight.

In [40]:
flights_df.where('tail_number IS NULL AND cancelled = 1').count()

14721

In [44]:
flights_df.where('departure_time IS NULL and cancelled = 1').count()

86153

#### Scheduled_time - 6 Nulls, 5 out of 6 flight were cancelled, 1 flight was diverted
These flights took off, but did not arrive in destination.
Scheduled_time is the estimated time in air?

In [28]:
flights_df.where('scheduled_time IS NULL').show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|year|month|day|day_of_week|airline|flight_number|tail_number|origin_airport|destination_airport|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

# Write to staging Iceberg Table

In [13]:
# Drop flights table
spark.sql('DROP TABLE airline.db.flights PURGE')

DataFrame[]

In [14]:
# Create flights table if not exists
spark.sql("""
CREATE TABLE IF NOT EXISTS airline.db.flights (
    year INT,
    month INT,
    day INT,
    day_of_week INT,
    airline STRING,
    flight_number INT,
    tail_number STRING,
    origin_airport STRING,
    destination_airport STRING,
    scheduled_departure INT,
    departure_time INT,
    departure_delay INT,
    taxi_out INT,
    wheels_off INT,
    scheduled_time INT,
    elapsed_time INT,
    air_time INT,
    distance INT,
    wheels_on INT,
    taxi_in INT,
    scheduled_arrival INT,
    arrival_time INT,
    arrival_delay INT,
    diverted INT,
    cancelled INT,
    cancellation_reason STRING,
    air_system_delay INT,
    security_delay INT,
    airline_delay INT,
    late_aircraft_delay INT,
    weather_delay INT
)
USING iceberg
PARTITIONED BY (month)
""")

DataFrame[]

In [15]:
# Write flights data
flights_df \
    .writeTo('airline.db.flights') \
    .append()

In [24]:
# # Create database if not exists
# spark.sql('CREATE DATABASE IF NOT EXISTS db')

# print(spark.catalog.tableExists('airline.db.flights'))

# # Create flights table if not exists
# if not spark.catalog.tableExists('airline.db.flights'):
#     spark.catalog.createTable(
#         'airline.db.flights',
#         schema=flights_schema,
#         source='iceberg',
#         partitioning='month'
#     )

# spark.catalog.getTable('airline.db.flights')

False


Table(name='flights', catalog='airline', namespace=['db'], description=None, tableType='MANAGED', isTemporary=False)

In [9]:
spark.sql('DESCRIBE EXTENDED airline.db.flights').show()

+-------------------+---------+-------+
|           col_name|data_type|comment|
+-------------------+---------+-------+
|               year|      int|   NULL|
|              month|      int|   NULL|
|                day|      int|   NULL|
|        day_of_week|      int|   NULL|
|            airline|   string|   NULL|
|      flight_number|      int|   NULL|
|        tail_number|   string|   NULL|
|     origin_airport|   string|   NULL|
|destination_airport|   string|   NULL|
|scheduled_departure|      int|   NULL|
|     departure_time|      int|   NULL|
|    departure_delay|      int|   NULL|
|           taxi_out|      int|   NULL|
|         wheels_off|      int|   NULL|
|     scheduled_time|      int|   NULL|
|       elapsed_time|      int|   NULL|
|           air_time|      int|   NULL|
|           distance|      int|   NULL|
|          wheels_on|      int|   NULL|
|            taxi_in|      int|   NULL|
+-------------------+---------+-------+
only showing top 20 rows



#### HIDDEN PARTITIONING!?
When partitioning on year, still getting 12 files in 1 partition.

Dataset came sorted by date, so hidden partitioning was automatically splitting into 12 parquet files?

When partitioning on month, what is the behavior?

In [45]:
metrics = spark.sql('select readable_metrics from airline.db.flights.files').collect()

# Check lower and upper bounds on month to ensure partitioning by month is being performed correctly
for metric in metrics:
    temp = metric.asDict()['readable_metrics'].asDict()['month']
    print(temp)

Row(column_size=1136, value_count=479230, null_value_count=0, nan_value_count=None, lower_bound=12, upper_bound=12)
Row(column_size=1136, value_count=469968, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=1)
Row(column_size=1225, value_count=503897, null_value_count=0, nan_value_count=None, lower_bound=6, upper_bound=6)
Row(column_size=1224, value_count=504312, null_value_count=0, nan_value_count=None, lower_bound=3, upper_bound=3)
Row(column_size=1182, value_count=496993, null_value_count=0, nan_value_count=None, lower_bound=5, upper_bound=5)
Row(column_size=1133, value_count=464946, null_value_count=0, nan_value_count=None, lower_bound=9, upper_bound=9)
Row(column_size=1179, value_count=485151, null_value_count=0, nan_value_count=None, lower_bound=4, upper_bound=4)
Row(column_size=1228, value_count=510536, null_value_count=0, nan_value_count=None, lower_bound=8, upper_bound=8)
Row(column_size=1271, value_count=520718, null_value_count=0, nan_value_count=None, lo

#### Checking table metadata

In [4]:
%%sql
SELECT
    *
FROM airline.db.flights.partitions

24/12/20 04:33:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


partition,spec_id,record_count,file_count,total_data_file_size_in_bytes,position_delete_record_count,position_delete_file_count,equality_delete_record_count,equality_delete_file_count,last_updated_at,last_updated_snapshot_id
Row(month=8),0,510536,1,11439099,0,0,0,0,2024-12-19 20:34:47.122000,1297397742035865824
Row(month=9),0,464946,1,10218131,0,0,0,0,2024-12-19 20:34:47.122000,1297397742035865824
Row(month=6),0,503897,1,11434535,0,0,0,0,2024-12-19 20:34:47.122000,1297397742035865824
Row(month=7),0,520718,1,11755348,0,0,0,0,2024-12-19 20:34:47.122000,1297397742035865824
Row(month=12),0,479230,1,10767738,0,0,0,0,2024-12-19 20:34:47.122000,1297397742035865824
Row(month=10),0,486165,1,10651549,0,0,0,0,2024-12-19 20:34:47.122000,1297397742035865824
Row(month=11),0,467972,1,10373628,0,0,0,0,2024-12-19 20:34:47.122000,1297397742035865824
Row(month=1),0,469968,1,10532934,0,0,0,0,2024-12-19 20:34:47.122000,1297397742035865824
Row(month=4),0,485151,1,10816223,0,0,0,0,2024-12-19 20:34:47.122000,1297397742035865824
Row(month=5),0,496993,1,11088572,0,0,0,0,2024-12-19 20:34:47.122000,1297397742035865824


In [32]:
%%sql
SELECT
    *
FROM airline.db.flights.files

+---------+--------------------------------------------------------------------------------------------------------+-------------+---------+---------------+--------------+--------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+---------------+--------------+---------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
%%sql
SELECT
    SUM(file_size_in_bytes)
FROM airline.db.flights.files

sum(file_size_in_bytes)
130081273


In [19]:
%%sql
SELECT 
    partition
FROM 
    airline.db.flights.files
WHERE 
    null_value_counts['7'] > 0
GROUP BY 
    partition

partition
Row(month=12)
Row(month=1)
Row(month=6)
Row(month=3)
Row(month=5)
Row(month=9)
Row(month=4)
Row(month=8)
Row(month=7)
Row(month=10)
